# 引用包

In [ ]:
import functools
import os
import json
import multiprocessing as mp
import re
import torch
from tqdm import tqdm
from torch_geometric.data import *
import threading
import networkx as nx
import math
# 这里只处理和分析了marple-1的数据，至于其他两个agent采集到的数据，涉及到的攻击较少，所以就不分析了
filePath="/home/monk/datasets/e5-clearscope/clearscope-2/"
filelist = os.listdir(filePath)

import hashlib
def stringtomd5(originstr):
    originstr = originstr.encode("utf-8")
    signaturemd5 = hashlib.sha256()
    signaturemd5.update(originstr)
    return signaturemd5.hexdigest() 

# 时间戳转换函数

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s


def ns_time_to_datetime_US(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int 秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return s

def datetime_to_ns_time(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)


In [ ]:
# ['ta1-clearscope-2-e5-official-1.bin.4.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.18.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.23.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.26.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.8.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.2.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.16.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.14.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.14.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.3.json',
#  'ta1-clearscope-2-e5-official-1.bin.33.json',
#  'ta1-clearscope-2-e5-official-1.bin.7.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.24.json',
#  'ta1-clearscope-2-e5-official-1.bin.11.json',
#  'ta1-clearscope-2-e5-official-1.bin.30.json',
#  'ta1-clearscope-2-e5-official-1.bin.29.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.22.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.18.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.15.json',
#  'ta1-clearscope-2-e5-official-1.bin.10.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.21.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.json',
#  'ta1-clearscope-2-e5-official-1.bin.3.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.25.json',
#  'ta1-clearscope-2-e5-official-1.bin.33.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.21.json',
#  'ta1-clearscope-2-e5-official-1.bin.25.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.1.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.30.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.16.json',
#  'ta1-clearscope-2-e5-official-1.bin.29.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.26.json',
#  'ta1-clearscope-2-e5-official-1.bin.25.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.13.json',
#  'ta1-clearscope-2-e5-official-1.bin.16.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.4.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.2.json',
#  'ta1-clearscope-2-e5-official-1.bin.32.json',
#  'ta1-clearscope-2-e5-official-1.bin.17.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.20.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.1.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.9.json',
#  'ta1-clearscope-2-e5-official-1.bin.10.json',
#  'ta1-clearscope-2-e5-official-1.bin.9.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.17.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.32.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.6.json',
#  'ta1-clearscope-2-e5-official-1.bin.26.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.10.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.14.json',
#  'ta1-clearscope-2-e5-official-1.bin.32.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.20.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.5.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.8.json',
#  'ta1-clearscope-2-e5-official-1.bin.23.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.9.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.3.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.4.json',
#  'ta1-clearscope-2-e5-official-1.bin.1.json',
#  'ta1-clearscope-2-e5-official-1.bin.27.json',
#  'ta1-clearscope-2-e5-official-1.bin.7.json',
#  'ta1-clearscope-2-e5-official-1.bin.17.json',
#  'ta1-clearscope-2-e5-official-1.bin.24.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.11.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.22.json',
#  'ta1-clearscope-2-e5-official-1.bin.12.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.24.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.6.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.23.json',
#  'ta1-clearscope-2-e5-official-1.bin.6.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.33.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.19.json',
#  'ta1-clearscope-2-e5-official-1.bin.13.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.28.json',
#  'ta1-clearscope-2-e5-official-1.bin.2.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.34.json',
#  'ta1-clearscope-2-e5-official-1.bin.28.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.20.json',
#  'ta1-clearscope-2-e5-official-1.bin.22.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.12.json',
#  'ta1-clearscope-2-e5-official-1.bin.31.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.27.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.19.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.5.json',
#  'ta1-clearscope-2-e5-official-1.bin.12.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.31.json',
#  'ta1-clearscope-2-e5-official-1.bin.11.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.15.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.31.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.8.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.29.json',
#  'ta1-clearscope-2-e5-official-1.bin.18.json',
#  'ta1-clearscope-2-e5-official-1.bin.19.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.15.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.7.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.13.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.28.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.5.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.21.json.1',
#  'ta1-clearscope-2-e5-official-1.bin.27.json.2',
#  'ta1-clearscope-2-e5-official-1.bin.30.json.2']

In [ ]:
# 时间戳转换函数

from datetime import datetime, timezone
import time
import pytz
from time import mktime
from datetime import datetime
import time
def ns_time_to_datetime(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    dt = datetime.fromtimestamp(int(ns) // 1000000000)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s


def ns_time_to_datetime_US(ns):
    """
    :param ns: int 纳秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00.000000000
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(ns) // 1000000000, tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    s += '.' + str(int(int(ns) % 1000000000)).zfill(9)
    return s

def time_to_datetime_US(s):
    """
    :param ns: int 秒时间戳
    :return: datetime   格式为 2013-10-10 23:40:00
    """
    tz = pytz.timezone('US/Eastern')
    dt = pytz.datetime.datetime.fromtimestamp(int(s), tz)
    s = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return s

def datetime_to_ns_time(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    timeStamp = int(time.mktime(timeArray))
    timeStamp = timeStamp * 1000000000
    return timeStamp

def datetime_to_ns_time_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp * 1000000000
    return int(timeStamp)

def datetime_to_timestamp_US(date):
    """
    :param date: str 格式为  %Y-%m-%d %H:%M:%S   例如 2013-10-10 23:40:00
    :return: 纳秒时间戳
    """
    tz = pytz.timezone('US/Eastern')
    timeArray = time.strptime(date, "%Y-%m-%d %H:%M:%S")
    dt = datetime.fromtimestamp(mktime(timeArray))
    timestamp = tz.localize(dt)
    timestamp=timestamp.timestamp()
    timeStamp = timestamp
    return int(timeStamp)


# 数据库

## 连接数据库

In [ ]:
# 连接postgessql 数据库，将数据存入数据库中
import psycopg2

from psycopg2 import extras as ex
connect = psycopg2.connect(database = 'tc_e5_clearscope_dataset_db',
                           user = 'postgres',
                           password = '21',
                           port = '5432'#一般是5432
                          )



# 创建一个cursor来执行数据库的操作
cur = connect.cursor()

In [ ]:
tt=cur.execute("""
    delete from subject_node_table where 1=1;
""")
print(tt)
connect.commit()  #需要手动提交

In [ ]:
#发生错误时需要回滚
connect.rollback()

## 数据库表

与之前一致

# 分别处理不同类型节点

## 套接字节点

In [ ]:
# 遍历数据集
netobjset=set()
netobj2hash={}# 
datalist=[]
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
#                 pass
                if "avro.cdm20.NetFlowObject" in line:
#                     print(line)
                    try:
                        res=re.findall('NetFlowObject":{"uuid":"(.*?)"(.*?)"localAddress":{"string":"(.*?)"},"localPort":{"int":(.*?)},"remoteAddress":{"string":"(.*?)"},"remotePort":{"int":(.*?)}',line)[0]

                        nodeid=res[0]
                        srcaddr=res[2]
                        srcport=res[3]
                        dstaddr=res[4]
                        dstport=res[5]

                        nodeproperty=srcaddr+","+srcport+","+dstaddr+","+dstport # 只关注向哪里发起的网络流 合理么？
#                         nodeproperty=dstaddr+","+dstport # 只关注向哪里发起的网络流 合理么？
                        hashstr=stringtomd5(nodeproperty)
                        netobj2hash[nodeid]=[hashstr,nodeproperty]
                        netobj2hash[hashstr]=nodeid
                        netobjset.add(hashstr)
                    except:
                            # 228786个文件节点是没有路径的，那么关于这种事件应该如何处理呢？ 直接扔掉?
                        pass
#                     print(match)


In [ ]:
datalist=[]
for i in netobj2hash.keys():
    if len(i)!=64:
        datalist.append([i]+[netobj2hash[i][0]]+netobj2hash[i][1].split(","))

#写入数据库


sql = '''insert into netflow_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交

del netobj2hash
del datalist

## 进程节点

In [ ]:
scusess_count=0
fail_count=0
subject_objset=set()
subject_obj2hash={}# 
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
#             for line in tqdm(f): 
            for line in (f):
                if "schema.avro.cdm20.Subject" in line:
#                     print(line)
                    subject_uuid=re.findall('avro.cdm20.Subject":{"uuid":"(.*?)",(.*?)"path":"(.*?)"',line)
#                
                    try:
#                         (subject_uuid[0][-1])
                        subject_obj2hash[subject_uuid[0][0]]=subject_uuid[0][-1]
                        scusess_count+=1
                    except:
                        try:
                            subject_obj2hash[subject_uuid[0][0]]="null"
                        except:
                            pass
#                             print(line)
#                         print(line)                        
                        fail_count+=1
                        

In [ ]:
datalist=[]
for i in subject_obj2hash.keys():
    if len(i)!=64:
        datalist.append([i]+[stringtomd5(subject_obj2hash[i]),subject_obj2hash[i]])
        
#写入数据库


sql = '''insert into subject_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交


del netobj2hash
del datalist

## 文件节点

In [ ]:

file_obj2hash={}
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in f:
                if "avro.cdm20.FileObject" in line:
#                     print(line)
                    
                    Object_uuid=re.findall('cdm20.FileObject":{"uuid":"(.*?)",(.*?){"map":{"path":"(.*?)"',line) 
#                     Object_uuid=re.findall('FileObject":{"uuid":"(.*?)",',line) 
                    try:
#                         file_node.add(Object_uuid[0])
                        file_obj2hash[Object_uuid[0][0]]=Object_uuid[0][2]
                    except:
                        print(line)

In [ ]:
datalist=[]
for i in file_obj2hash.keys():
    if len(i)!=64 :
        datalist.append([i]+[stringtomd5(file_obj2hash[i]),file_obj2hash[i]])

# 写入数据库
sql = '''insert into file_node_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist,page_size=10000)
connect.commit()  #需要手动提交
del file_obj2hash
del datalist

## 处理事件数据

### 预先准备

In [ ]:

# 生成node2id表的数据
node_list={}
##################################################################################################
sql="""
select * from file_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:    
    node_list[i[1]]=["file",i[-1]]

file_uuid2hash={}
for i in records:
    file_uuid2hash[i[0]]=i[1]
##################################################################################################    
sql="""
select * from subject_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:
    node_list[i[1]]=["subject",i[-1]]

subject_uuid2hash={}
for i in records:
    subject_uuid2hash[i[0]]=i[1]
##################################################################################################
sql="""
select * from netflow_node_table;
"""
cur.execute(sql)
records = cur.fetchall()

for i in records:
    
    node_list[i[1]]=["netflow",i[-2]+":"+i[-1]]

net_uuid2hash={}
for i in records:
    net_uuid2hash[i[0]]=i[1]

In [ ]:
node_list_database=[]
node_index=0
for i in node_list:
    node_list_database.append([i]+node_list[i]+[node_index])
    node_index+=1

In [ ]:
#写入数据库
sql = '''insert into node2id
                     values %s
        '''
ex.execute_values(cur,sql, node_list_database,page_size=10000)
connect.commit()  #需要手动提交

In [ ]:
# 构建nodeid 与msg之间的构建  创建字典变量 nodeid2msg
sql="select * from node2id ORDER BY index_id;"
cur.execute(sql)
rows = cur.fetchall()

nodeid2msg={}  # 可以实现 nodeid 转 msg      node hash 转 nodeid
for i in rows:
    nodeid2msg[i[0]]=i[-1]
    nodeid2msg[i[-1]]={i[1]:i[2]}  #0到 n-1

In [ ]:
nodeid2msg

### 开始处理

In [ ]:
include_edge_type=[
    'EVENT_CLOSE',
    'EVENT_OPEN',
    'EVENT_READ',
    'EVENT_WRITE',
     'EVENT_EXECUTE',
    'EVENT_RECVFROM',
    'EVENT_RECVMSG',
    'EVENT_SENDMSG',
    'EVENT_SENDTO',
]

In [ ]:
datalist=[]
edge_type=set()
reverse=["EVENT_READ","EVENT_RECVFROM","EVENT_RECVMSG"]        
for file in tqdm(filelist):
        with open(filePath + file, "r") as f:
            for line in (f):
                if '{"datum":{"com.bbn.tc.schema.avro.cdm20.Event"' in line:
#                     print(line)
                    subject_uuid=re.findall('"subject":{"com.bbn.tc.schema.avro.cdm20.UUID":"(.*?)"}',line)
                    predicateObject_uuid=re.findall('"predicateObject":{"com.bbn.tc.schema.avro.cdm20.UUID":"(.*?)"}',line)
                    if len(subject_uuid) >0 and len(predicateObject_uuid)>0:
                        if subject_uuid[0] in subject_uuid2hash\
                        and (predicateObject_uuid[0] in file_uuid2hash or predicateObject_uuid[0] in net_uuid2hash):
                            #开始处理事件数据
                            relation_type=re.findall('"type":"(.*?)"',line)[0]
                            time_rec=re.findall('"timestampNanos":(.*?),',line)[0]
                            time_rec=int(time_rec) # 将时间转成秒为单位
                            subjectId=subject_uuid2hash[subject_uuid[0]]
                            if predicateObject_uuid[0] in file_uuid2hash:
                                objectId=file_uuid2hash[predicateObject_uuid[0]]
                            else:
                                objectId=net_uuid2hash[predicateObject_uuid[0]]
#                                 print(line)
                            edge_type.add(relation_type)
                            if relation_type in reverse:
                                datalist.append([objectId,nodeid2msg[objectId],relation_type,subjectId,nodeid2msg[subjectId],time_rec])
                            else:
                                datalist.append([subjectId,nodeid2msg[subjectId],relation_type,objectId,nodeid2msg[objectId],time_rec])

               
     
                     

In [1]:
rel=filter_type=[
   'EVENT_ACCEPT',
 'EVENT_CLONE',
 'EVENT_CLOSE',
 'EVENT_CREATE_OBJECT',
 'EVENT_EXECUTE',
 'EVENT_OPEN',
 'EVENT_READ',
 'EVENT_RECVFROM',
 'EVENT_SENDTO',
 'EVENT_WRITE',
]


In [ ]:
# 按照event type 筛选出需要用的events
datalist_new=[]
for d in datalist:
    if d[2] in filter_type:      
        datalist_new.append(d)

In [ ]:
#写入数据库
#事件数据插入数据库

sql = '''insert into event_table
                     values %s
        '''
ex.execute_values(cur,sql, datalist_new,page_size=10000)
connect.commit()  #需要手动提交

# 属性编码

In [ ]:
from sklearn.feature_extraction import FeatureHasher
from torch_geometric.transforms import NormalizeFeatures

from sklearn import preprocessing #导入用于数据标准化的模块
import numpy as np
# 将path 转成分层的list

FH_string=FeatureHasher(n_features=16,input_type="string")
FH_dict=FeatureHasher(n_features=16,input_type="dict")


def path2higlist(p):
    l=[]
    spl=p.strip().split('/')
    for i in spl:
        if len(l)!=0:
            l.append(l[-1]+'/'+i)
        else:
            l.append(i)
#     print(l)
    return l

def ip2higlist(p):
    l=[]
    spl=p.strip().split('.')
    for i in spl:
        if len(l)!=0:
            l.append(l[-1]+'.'+i)
        else:
            l.append(i)
#     print(l)
    return l


def subject2higlist(p):
    l=[]
    spl=p.strip().split('/')
    for i in spl:
        if len(l)!=0:
            l.append(l[-1]+'/'+i)
        else:
            l.append(i)
#     print(l)
    return l


def list2str(l):
    s=''
    for i in l:
        s+=i
    return s

In [ ]:
node_msg_vec=[]
node_msg_dic_list=[]
for i in tqdm(nodeid2msg.keys()):
    if type(i)==int:
        if 'netflow' in nodeid2msg[i].keys():
            higlist=['netflow']
            higlist+=ip2higlist(nodeid2msg[i]['netflow'])
            
        if 'file' in nodeid2msg[i].keys():
            higlist=['file']
            higlist+=path2higlist(nodeid2msg[i]['file'])
            
#             print(higlist)
        if 'subject' in nodeid2msg[i].keys():
            higlist=['subject']
            higlist+=subject2higlist(nodeid2msg[i]['subject'])
        node_msg_dic_list.append(list2str(higlist))

In [ ]:
node2higvec=[]
for i in tqdm(node_msg_dic_list):
    vec=FH_string.transform([i]).toarray()
    node2higvec.append(vec)
    
#100%|██████████| 139960/139960 [00:16<00:00, 8724.76it/s]

In [ ]:
node2higvec=np.array(node2higvec).reshape([-1,16])

In [ ]:
rel2id={1: 'EVENT_ACCEPT',
 'EVENT_ACCEPT': 1,
 2: 'EVENT_CLONE',
 'EVENT_CLONE': 2,
 3: 'EVENT_CLOSE',
 'EVENT_CLOSE': 3,
 4: 'EVENT_CREATE_OBJECT',
 'EVENT_CREATE_OBJECT': 4,
 5: 'EVENT_EXECUTE',
 'EVENT_EXECUTE': 5,
 6: 'EVENT_OPEN',
 'EVENT_OPEN': 6,
 7: 'EVENT_READ',
 'EVENT_READ': 7,
 8: 'EVENT_RECVFROM',
 'EVENT_RECVFROM': 8,
 9: 'EVENT_SENDTO',
 'EVENT_SENDTO': 9,
 10: 'EVENT_WRITE',
 'EVENT_WRITE': 10}

In [ ]:
# 生成edge type one-hot变量
relvec=torch.nn.functional.one_hot(torch.arange(0, len(rel2id.keys())//2), num_classes=len(rel2id.keys())//2)

In [ ]:
# 将不同的relation type转为 变量 rel2vec   通过该变量，可以将字符串的关系类型转成ont hot编码
rel2vec={}
for i in rel2id.keys():
    if type(i) is not int:
        rel2vec[i]= relvec[rel2id[i]-1]
        rel2vec[relvec[rel2id[i]-1]]=i

In [ ]:
## 保存结果变量
torch.save(node2higvec,"node2higvec")
torch.save(rel2vec,"rel2vec")

# 生成数据集

In [ ]:
# 加载节点特征字典，边特征字典数据
node2higvec=torch.load("./node2higvec")
rel2vec=torch.load("./rel2vec")

In [ ]:
for day in tqdm(range(8,18)):
    start_timestamp=datetime_to_ns_time_US('2019-05-'+str(day)+' 00:00:00')
    end_timestamp=datetime_to_ns_time_US('2019-05-'+str(day+1)+' 00:00:00')
    sql="""
    select * from event_table
    where
          timestamp_rec>'%s' and timestamp_rec<'%s'
           ORDER BY timestamp_rec;
    """%(start_timestamp,end_timestamp)
    cur.execute(sql)
    events = cur.fetchall()
    print('2019-05-'+str(day)," events count:",str(len(events)))
    edge_list=[]
    for e in events:
        edge_temp=[int(e[1]),int(e[4]),e[2],e[5]]
        if e[2] in rel2id:# 如果该边类型符合 过滤器，就加入到数据集中
#         if True:
            edge_list.append(edge_temp)
    print('2019-05-'+str(day)," edge list len:",str(len(edge_list)))
    # 保存数据集数据
    dataset = TemporalData()
    src = []
    dst = []
    msg = []
    t = []
    for i in edge_list:
        src.append(int(i[0]))
        dst.append(int(i[1]))
    #     msg.append(torch.cat([torch.from_numpy(node2higvec_bn[i[0]]), rel2vec[i[2]], torch.from_numpy(node2higvec_bn[i[1]])] ))
        msg.append(torch.cat([torch.from_numpy(node2higvec[i[0]]), rel2vec[i[2]], torch.from_numpy(node2higvec[i[1]])] ))
        t.append(int(i[3]))
    if len(edge_list)>0:
        dataset.src = torch.tensor(src)
        dataset.dst = torch.tensor(dst)
        dataset.t = torch.tensor(t)
        dataset.msg = torch.vstack(msg)
        dataset.src = dataset.src.to(torch.long)
        dataset.dst = dataset.dst.to(torch.long)
        dataset.msg = dataset.msg.to(torch.float)
        dataset.t = dataset.t.to(torch.long)
        torch.save(dataset, "./temporal_graphs/graph_5_"+str(day)+".TemporalData.simple")  